The first thing that we need to do is import the relevant packages. I will import the machine learning packages at a later stage.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

The next thing that I will do is import the training data and have a look at it. I have used the generic name "df" to call the data.

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


There are 81 columns in the dataset which is obviously a very large number. I will initially create a model that only uses a few features and then iterate from there to create a more sophisticated model. These features will be numerical columns because then I do not need to spend time cleaning or incoding the data. The target variable will obviously be the `SalePrice` column.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [5]:
X = df[['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt']]
y = df['SalePrice']

neigh_dummies = pd.get_dummies(df.Neighborhood, prefix = "Neighbourhood", drop_first=True)
X = pd.concat([X, neigh_dummies], axis = 1)

zone_dummies = pd.get_dummies(df.MSZoning, prefix = "MSZoning", drop_first=True)
X = pd.concat([X, zone_dummies], axis = 1)


In [6]:
X.fillna(X.mean(), inplace=True)

In [7]:
y.isna().sum()

0

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [9]:
from sklearn.preprocessing import MinMaxScaler
mm_X = MinMaxScaler(feature_range=(0, 1))
X_train = mm_X.fit_transform(X_train)
X_test = mm_X.transform(X_test)

"""
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
""";

In [10]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.235294,0.041534,0.666667,0.50,0.869565,0.700000,0.119419,0.0,0.032534,0.122750,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.235294,0.050008,0.777778,0.50,0.949275,0.900000,0.000000,0.0,0.505565,0.193290,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.235294,0.057108,0.888889,0.50,0.985507,0.983333,0.000000,0.0,0.824486,0.315221,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.294118,0.047372,0.777778,0.75,0.413043,0.000000,0.000000,0.0,0.339041,0.129624,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.176471,0.038973,0.555556,0.75,0.340580,0.666667,0.000000,0.0,0.344178,0.131588,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Running the Model

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [12]:
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(.2))

model.add(Dense(1))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.compile(loss='mean_squared_error',optimizer='adam',
                metrics=['mean_squared_error', 'mean_absolute_error'])

Instructions for updating:
Use tf.cast instead.


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 8,385
Trainable params: 8,385
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(X_train, y_train, epochs=800, validation_split=.2, batch_size=32)

Train on 1109 samples, validate on 278 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/800
1109/1109 [==============================] - 1s 605us/sample - loss: 38524007814.5789 - mean_squared_error: 38524014592.0000 - mean_absolute_error: 179800.0938 - val_loss: 39644552567.7122 - val_mean_squared_error: 39644549120.0000 - val_mean_absolute_error: 181830.7969
Epoch 2/800
1109/1109 [==============================] - 0s 87us/sample - loss: 38518310936.0072 - mean_squared_error: 38518308864.0000 - mean_absolute_error: 179785.0156 - val_loss: 39632596461.5827 - val_mean_squared_error: 39632596992.0000 - val_mean_absolute_error: 181799.8594
Epoch 3/800
1109/1109 [==============================] - 0s 84us/sample - loss: 38495756239.9856 - mean_squared_error: 38495760384.0000 - mean_absolute_error: 179725.2031 - val_loss: 39594016944.8058 - val_mean_squared_error: 39594016768.0000 - val_mean_absolute_error: 181699.9375
Epoch 4/800
1109/1109 [==============================] - 0

Epoch 29/800
1109/1109 [==============================] - 0s 80us/sample - loss: 6565272164.6456 - mean_squared_error: 6565272576.0000 - mean_absolute_error: 55146.8906 - val_loss: 6429703344.8058 - val_mean_squared_error: 6429703168.0000 - val_mean_absolute_error: 53516.5625
Epoch 30/800
1109/1109 [==============================] - 0s 80us/sample - loss: 5810226371.9820 - mean_squared_error: 5810226688.0000 - mean_absolute_error: 50032.3750 - val_loss: 5625655458.0719 - val_mean_squared_error: 5625655296.0000 - val_mean_absolute_error: 48089.1133
Epoch 31/800
1109/1109 [==============================] - 0s 85us/sample - loss: 5299100723.2462 - mean_squared_error: 5299100672.0000 - mean_absolute_error: 46479.9375 - val_loss: 5013349202.8777 - val_mean_squared_error: 5013349376.0000 - val_mean_absolute_error: 43758.3164
Epoch 32/800
1109/1109 [==============================] - 0s 81us/sample - loss: 4714456441.9982 - mean_squared_error: 4714456064.0000 - mean_absolute_error: 43097.9453 

1109/1109 [==============================] - 0s 80us/sample - loss: 3215650893.1001 - mean_squared_error: 3215650816.0000 - mean_absolute_error: 38083.2422 - val_loss: 2738231284.9496 - val_mean_squared_error: 2738231296.0000 - val_mean_absolute_error: 34275.5000
Epoch 59/800
1109/1109 [==============================] - 0s 77us/sample - loss: 3088484018.6691 - mean_squared_error: 3088483840.0000 - mean_absolute_error: 37718.1289 - val_loss: 2716828756.7194 - val_mean_squared_error: 2716828672.0000 - val_mean_absolute_error: 34132.6875
Epoch 60/800
1109/1109 [==============================] - 0s 83us/sample - loss: 2986671438.2543 - mean_squared_error: 2986670848.0000 - mean_absolute_error: 36971.7500 - val_loss: 2698915554.5324 - val_mean_squared_error: 2698915584.0000 - val_mean_absolute_error: 33916.0234
Epoch 61/800
1109/1109 [==============================] - 0s 80us/sample - loss: 3132145069.0135 - mean_squared_error: 3132144896.0000 - mean_absolute_error: 37557.7891 - val_loss: 2

Epoch 88/800
1109/1109 [==============================] - 0s 81us/sample - loss: 2691429947.7872 - mean_squared_error: 2691429888.0000 - mean_absolute_error: 33773.5000 - val_loss: 2212118547.3381 - val_mean_squared_error: 2212118528.0000 - val_mean_absolute_error: 29415.6582
Epoch 89/800
1109/1109 [==============================] - 0s 84us/sample - loss: 2682792782.7160 - mean_squared_error: 2682792448.0000 - mean_absolute_error: 33984.8750 - val_loss: 2187984222.8489 - val_mean_squared_error: 2187984128.0000 - val_mean_absolute_error: 29489.4473
Epoch 90/800
1109/1109 [==============================] - 0s 82us/sample - loss: 2621661590.7376 - mean_squared_error: 2621661184.0000 - mean_absolute_error: 33657.2578 - val_loss: 2166872926.8489 - val_mean_squared_error: 2166872832.0000 - val_mean_absolute_error: 29519.8320
Epoch 91/800
1109/1109 [==============================] - 0s 82us/sample - loss: 2757106475.0514 - mean_squared_error: 2757106176.0000 - mean_absolute_error: 34333.3438 

1109/1109 [==============================] - 0s 90us/sample - loss: 2261840666.4310 - mean_squared_error: 2261840896.0000 - mean_absolute_error: 31128.9297 - val_loss: 1786770055.3669 - val_mean_squared_error: 1786770048.0000 - val_mean_absolute_error: 26037.1387
Epoch 118/800
1109/1109 [==============================] - 0s 93us/sample - loss: 2202776367.3219 - mean_squared_error: 2202776320.0000 - mean_absolute_error: 31395.6426 - val_loss: 1780031561.6691 - val_mean_squared_error: 1780031488.0000 - val_mean_absolute_error: 25810.3145
Epoch 119/800
1109/1109 [==============================] - 0s 92us/sample - loss: 2158163270.4058 - mean_squared_error: 2158163200.0000 - mean_absolute_error: 30281.4277 - val_loss: 1761654630.6763 - val_mean_squared_error: 1761654656.0000 - val_mean_absolute_error: 25823.2285
Epoch 120/800
1109/1109 [==============================] - 0s 97us/sample - loss: 2261926834.4382 - mean_squared_error: 2261926912.0000 - mean_absolute_error: 30938.0703 - val_loss

...

Epoch 681/800
1109/1109 [==============================] - 0s 100us/sample - loss: 1310544019.1596 - mean_squared_error: 1310543744.0000 - mean_absolute_error: 25085.3066 - val_loss: 926853425.9568 - val_mean_squared_error: 926853376.0000 - val_mean_absolute_error: 19271.9531
Epoch 682/800
1109/1109 [==============================] - 0s 98us/sample - loss: 1571661689.3057 - mean_squared_error: 1571661952.0000 - mean_absolute_error: 26219.0312 - val_loss: 928284855.2518 - val_mean_squared_error: 928284928.0000 - val_mean_absolute_error: 19273.3594
Epoch 683/800
1109/1109 [==============================] - 0s 90us/sample - loss: 1483123009.0965 - mean_squared_error: 1483123200.0000 - mean_absolute_error: 24805.8809 - val_loss: 929006218.1295 - val_mean_squared_error: 929006208.0000 - val_mean_absolute_error: 19266.4512
Epoch 684/800
1109/1109 [==============================] - 0s 99us/sample - loss: 1414423251.9675 - mean_squared_error: 1414423040.0000 - mean_absolute_error: 24965.8359 -

Epoch 711/800
1109/1109 [==============================] - 0s 92us/sample - loss: 1371687030.9973 - mean_squared_error: 1371687296.0000 - mean_absolute_error: 24482.7305 - val_loss: 920573321.2086 - val_mean_squared_error: 920573376.0000 - val_mean_absolute_error: 19196.1797
Epoch 712/800
1109/1109 [==============================] - 0s 91us/sample - loss: 1341993571.6646 - mean_squared_error: 1341993728.0000 - mean_absolute_error: 24864.6172 - val_loss: 919585119.3094 - val_mean_squared_error: 919585152.0000 - val_mean_absolute_error: 19192.5957
Epoch 713/800
1109/1109 [==============================] - 0s 96us/sample - loss: 1417609763.5491 - mean_squared_error: 1417609728.0000 - mean_absolute_error: 25310.4824 - val_loss: 914936777.8993 - val_mean_squared_error: 914936768.0000 - val_mean_absolute_error: 19230.6660
Epoch 714/800
1109/1109 [==============================] - 0s 91us/sample - loss: 1337087014.5500 - mean_squared_error: 1337086976.0000 - mean_absolute_error: 24244.0020 - 

Epoch 741/800
1109/1109 [==============================] - 0s 93us/sample - loss: 1388464246.3048 - mean_squared_error: 1388464256.0000 - mean_absolute_error: 26003.3711 - val_loss: 926123112.9784 - val_mean_squared_error: 926123072.0000 - val_mean_absolute_error: 19158.6973
Epoch 742/800
1109/1109 [==============================] - 0s 94us/sample - loss: 1292492432.6204 - mean_squared_error: 1292492544.0000 - mean_absolute_error: 23848.7891 - val_loss: 918659824.3453 - val_mean_squared_error: 918659840.0000 - val_mean_absolute_error: 19103.6543
Epoch 743/800
1109/1109 [==============================] - 0s 92us/sample - loss: 1291803017.2913 - mean_squared_error: 1291802752.0000 - mean_absolute_error: 24058.1191 - val_loss: 917887404.6619 - val_mean_squared_error: 917887360.0000 - val_mean_absolute_error: 19107.1309
Epoch 744/800
1109/1109 [==============================] - 0s 95us/sample - loss: 1422709521.4860 - mean_squared_error: 1422709632.0000 - mean_absolute_error: 24346.0195 - 

1109/1109 [==============================] - 0s 94us/sample - loss: 1446838773.8431 - mean_squared_error: 1446838784.0000 - mean_absolute_error: 25270.0742 - val_loss: 913339038.3885 - val_mean_squared_error: 913339072.0000 - val_mean_absolute_error: 19065.7520
Epoch 771/800
1109/1109 [==============================] - 0s 94us/sample - loss: 1443597644.4364 - mean_squared_error: 1443597568.0000 - mean_absolute_error: 24754.7773 - val_loss: 918397519.1942 - val_mean_squared_error: 918397504.0000 - val_mean_absolute_error: 19080.6934
Epoch 772/800
1109/1109 [==============================] - 0s 95us/sample - loss: 1411407069.4896 - mean_squared_error: 1411407232.0000 - mean_absolute_error: 24841.3867 - val_loss: 912056862.3885 - val_mean_squared_error: 912056832.0000 - val_mean_absolute_error: 19057.4570
Epoch 773/800
1109/1109 [==============================] - 0s 94us/sample - loss: 1366656383.6537 - mean_squared_error: 1366656384.0000 - mean_absolute_error: 24766.6289 - val_loss: 9113

Epoch 800/800
1109/1109 [==============================] - 0s 96us/sample - loss: 1402586929.6303 - mean_squared_error: 1402586880.0000 - mean_absolute_error: 24903.2246 - val_loss: 909175226.7050 - val_mean_squared_error: 909175296.0000 - val_mean_absolute_error: 19085.9434


In [16]:
y_pred = model.predict(X_test)

In [17]:
print("Mean Squared Error:", mean_squared_error(np.log(y_test), np.log(y_pred)))
print('Root Mean Squared Error:', mean_squared_error(np.log(y_test), np.log(y_pred)) **.5)

Mean Squared Error: 0.037884331639481586
Root Mean Squared Error: 0.19463897769840857


I think that I have finally got a model that makes somewhat decent predictions. There is still a lot of room for improvement but as a first attempt it probably is not too bad. Let us see how it performs.

In [18]:
testdf = pd.read_csv('test.csv')

In [19]:
X_test = testdf[['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt']]


neigh_dummies = pd.get_dummies(testdf.Neighborhood, prefix = "Neighbourhood", drop_first=True)
X_test = pd.concat([X_test, neigh_dummies], axis = 1)


zone_dummies = pd.get_dummies(testdf.MSZoning, prefix = "MSZoning", drop_first=True)
X_test = pd.concat([X_test, zone_dummies], axis = 1)
X_test = mm_X.transform(X_test)

In [20]:
X_test = pd.DataFrame(X_test)

In [21]:
X_test.fillna(X_test.mean(), inplace=True)

In [22]:
predictions = model.predict(X_test.values)

In [23]:
submission = pd.DataFrame({'Id': testdf['Id'].values,
              'SalePrice': predictions.ravel()})

In [24]:
submission.to_csv('housingpricesv7', index = False)

In [25]:
submission.tail()

,Id,SalePrice
1454,2915,76197.914062
1455,2916,90633.179688
1456,2917,186071.062500
1457,2918,97323.593750
1458,2919,224217.125000
